In [ ]:
# Data : 
#     +) https://www.kaggle.com/datasets/canomercik/wider-face-dataset-for-yolov12-format

In [ ]:
!pip install ultralytics
!pip install -U albumentations
!pip install numpy==1.26.4 scipy==1.13.0 --force-reinstall

In [ ]:
import os
import numpy as np
import torch
import scipy
from scipy.signal import savgol_filter
from ultralytics import YOLO

# Load model - Train - Evaluate

In [ ]:
def main():

    # 1. Chọn model
    model = YOLO('yolo11n.pt') 

    # 2. Training
    results = model.train(
        data='/kaggle/input/wider-face-dataset-for-yolov12-format/data.yaml',
        
        device="cuda",
        
        # --- Image Size ---
        imgsz=640,
        rect=False,
        
        # --- Cấu hình Training ---
        epochs=120,
        batch=-1,       
        workers=8,
        
        # --- Optimizer ---
        optimizer='SGD',
        lr0=0.01,
        lrf=0.01,
        momentum=0.937,
        weight_decay=0.0005,
        
        # --- Augmentation ---
        augment=True,
        mosaic=1.0,
        close_mosaic=10,
        
        hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
        fliplr=0.5,
        
        # --- Tinh chỉnh Loss ---
        box=7.5,
        
        # --- Các tham số khác ---
        cos_lr=True,
        patience=30,
        seed=42,
        name='yolov11n-face-640-finetune',
        verbose=True,
        exist_ok=True,
    )

    # 3. Đánh giá model
    print("Training completed. Validating best model...")
    
    # Load lại best weights
    best_model_path = 'runs/detect/yolov11n-face-640-finetune/weights/best.pt'
    
    # Kiểm tra xem file có tồn tại
    if os.path.exists(best_model_path):
        best_model = YOLO(best_model_path)
        
        metrics = best_model.val(
            split='val', 
            imgsz=640, 
            rect=True,       
            device="cuda"
        )
        
        print(f"mAP@50: {metrics.box.map50}")
        print(f"mAP@50-95: {metrics.box.map}")
    else:
        print("Không tìm thấy file best.pt. Có thể quá trình train chưa hoàn tất.")

if __name__ == '__main__':
    main()

# Download runs.zip

In [ ]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))
    
download_file('/kaggle/working/runs', 'runs') 